In [1]:
# imports
from sqlalchemy import create_engine, Column, String, Integer, Float, Date, MetaData, Table, type_coerce
from sqlalchemy.orm import mapper, create_session
import csv
from sqlalchemy import case
import numpy as np
from sqlalchemy.ext.automap import automap_base
import matplotlib.pyplot as plt
import sqlalchemy.types as types
import numbers
from sqlalchemy.sql import and_, or_, not_
from sqlalchemy.sql import select
import datetime as dt
%matplotlib inline

In [2]:
# definitions
file1='/ocean/eolson/MEOPAR/obs/BioChem/BioChem_Query_1610/BioChem_Query_1610_Data.csv'

In [3]:
engine = create_engine('sqlite:////ocean/eolson/MEOPAR/obs/BioChem/BioChemQueryDB.sqlite')

In [4]:
table = None
metadata = MetaData(bind=engine)


In [5]:
class forceNumeric(types.TypeDecorator):

    impl = types.Float

    def process_bind_param(self, value, dialect):
        try:
            int(float(value))
        except:
            value = None
        return value

class forceInt(types.TypeDecorator):
    
    impl = types.Integer
    def process_bind_param(self, value, dialect):
        try:
            int(value)
        except:
            value = None
        return value
    
class toDate(types.TypeDecorator):
    
    impl = types.Date
    def process_bind_param(self, value, dialect):
        try:
            #monDict={'Jan':1,'Feb':2,'Mar':3,'Apr':4, 'May':5, 'Jun':6,
            #         'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
            monDict={'Jan':'01','Feb':'02','Mar':'03','Apr':'04', 'May':'05', 'Jun':'06',
                     'Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}
            year=value[-4:]
            month=monDict[value[-8:-5]]
            day=value[0:2]
            value=dt.date(int(year),int(month), int(day))
        except:
            value = None
        return value

class toHour(types.TypeDecorator):
    
    impl = types.Float
    def process_bind_param(self, value, dialect):
        try:
            mm=float(value[-2:])
            hh=float(value[0:-2])
            value=hh+mm/60.0
        except:
            value = None
        return value

In [6]:
def coltype(ikey):
    typedict = {
        'DATA_CENTER': String(),
        'DESCRIPTOR-NAME': String(),
        'AREA_NAME': String(),
        'COLLECTOR_STATION_NAME': String(),
        'START_DATE': toDate(),
        'START_TIME': toHour(),
        'COLLECTOR_DEPLOYMENT_ID': String(),
        'GEAR_MODEL (GEAR_SIZE)': String(),
        'Silicate_datatype': String(),
        'Phosphate_datatype': String(),
        'Nitrate_datatype': String(),
    }
    return typedict.get(ikey, forceNumeric())
    

In [7]:
with open(file1) as f:
    cf = csv.DictReader(f, delimiter=',')
    for row in cf:
        if table is None:
            table = Table('profiles', metadata, Column('id', Integer, primary_key=True),
                *(Column(rowname, coltype(rowname)) for rowname in row.keys()))
            table.create()
        table.insert().values(**row).execute()
class Profs(object): pass
mapper(Profs, table)

<Mapper at 0x7f44d33465f8; Profs>

In [8]:
Base = automap_base()
# reflect the tables in salish.sqlite:
Base.prepare(engine, reflect=True)
# mapped classes have been created
Profs=Base.classes.profiles
session = create_session(bind = engine, autocommit = False, autoflush = True)


In [9]:
print(row.keys())

dict_keys(['Silicate', 'DESCRIPTOR-NAME', 'COLLECTOR_SAMPLE_ID', 'END_DEPTH', 'START_TIME', 'Silicate_qcCode', 'QRY_NO', 'Nitrate_datatype', 'COLLECTOR_EVENT_ID', 'Silicate_datatype', 'DATA_CENTER', 'SOUNDING', 'Phosphate_qcCode', 'COLLECTOR (RESPONSIBLE_GROUP)', 'Phosphate', 'COLLECTOR_DEPLOYMENT_ID', 'START_DEPTH', 'Phosphate_datatype', 'START_LON', 'GEAR_MODEL (GEAR_SIZE)', 'Nitrate', 'COLLECTOR_STATION_NAME', 'START_DATE', 'START_LAT', 'Nitrate_qcCode', 'AREA_NAME'])


In [11]:
data=session.query(Profs.START_DATE,Profs.START_TIME, Profs.START_DEPTH,Profs.START_LAT,Profs.START_LON, 
                  Profs.Nitrate, Profs.Silicate, Profs.Phosphate
                  ).filter(or_(
                           Profs.Nitrate != None,
                           Profs.Phosphate != None,
                           Profs.Silicate != None,
                          )).all()
for row in data:
    print(row)

(datetime.date(2009, 5, 2), 2.15, 174.14, 48.65333, -123.50667, None, 96.2, 3.72)
(datetime.date(2009, 5, 2), 2.15, 5.06, 48.65333, -123.50667, 0.3, 3.1, 0.37)
(datetime.date(2009, 5, 2), 2.15, 0.89, 48.65333, -123.50667, 1.1, 4.0, 0.33)
(datetime.date(2009, 5, 2), 2.15, 149.37, 48.65333, -123.50667, None, 97.9, 4.75)
(datetime.date(2009, 5, 2), 2.15, 124.3, 48.65333, -123.50667, 19.1, 77.5, 3.28)
(datetime.date(2009, 5, 2), 2.15, 99.92, 48.65333, -123.50667, 26.5, 62.8, 2.18)
(datetime.date(2009, 5, 2), 2.15, 74.55, 48.65333, -123.50667, 28.7, 59.0, 2.52)
(datetime.date(2009, 5, 2), 2.15, 49.67, 48.65333, -123.50667, 26.3, 47.7, 2.38)
(datetime.date(2009, 5, 2), 2.15, 30.44, 48.65333, -123.50667, 17.8, 29.7, 1.84)
(datetime.date(2009, 5, 5), 19.666666666666668, 109.74, 48.65333, -123.505, 26.1, 65.4, 2.37)
(datetime.date(2009, 5, 5), 19.666666666666668, 89.72, 48.65333, -123.505, 22.4, 85.2, 2.54)
(datetime.date(2009, 5, 5), 19.666666666666668, 30.04, 48.65333, -123.505, 18.1, 32.4, 1

In [12]:
session.close()
engine.dispose()